In [1]:
import os, json, pandas as pd, traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv

load_dotenv() 

True

In [4]:
import os
from getpass import getpass

# os.environ["AI21_API_KEY"] = getpass()
os.environ["COHERE_API_KEY"] = getpass()

In [5]:
# from langchain_ai21 import AI21LLM
# # llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.3)
# llm = AI21LLM(model="j2-ultra")
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage
llm = ChatCohere(model="command", max_tokens=256, temperature=0.75)

In [6]:
llm

ChatCohere(client=<cohere.client.Client object at 0x000001DECB627FD0>, async_client=<cohere.client.AsyncClient object at 0x000001DECB661040>, model='command', temperature=0.75, cohere_api_key=SecretStr('**********'))

In [7]:

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
from langchain_ai21 import AI21LLM
import PyPDF2

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [9]:

TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [11]:
quiz_chain =LLMChain(llm=llm , prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [12]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)


In [14]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)


In [16]:

file_path=r"E:\ProexeLancers\GenAI_LLMS\MCQGenerator\experiments\data.txt"

with open(file_path, 'r') as file:
    TEXT = file.read()

print(TEXT)

Title: The Synergy of Generative AI and Large Language Models: A Paradigm Shift in Creativity and Understanding

Introduction:
In recent years, the integration of Generative Artificial Intelligence (AI) and Large Language Models (LLMs) has led to groundbreaking advancements across various fields. This essay explores the symbiotic relationship between these two technologies, delving into their individual capabilities and the transformative impact they have when combined. Through examining their applications in creative endeavors, problem-solving, and knowledge dissemination, it becomes evident that their synergy heralds a new era of innovation and comprehension.

Understanding Generative AI and Large Language Models:
Generative AI refers to a subset of artificial intelligence that focuses on generating new content autonomously. It encompasses a wide range of techniques, from rule-based systems to deep learning models like Generative Adversarial Networks (GANs) and Variational Autoencode

In [17]:

# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [18]:
NUMBER=5 
SUBJECT="Artificial Intellegence"
TONE="simple"

In [19]:

response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

e:\ProexeLancers\GenAI_LLMS\MCQGenerator\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Title: The Synergy of Generative AI and Large Language Models: A Paradigm Shift in Creativity and Understanding

Introduction:
In recent years, the integration of Generative Artificial Intelligence (AI) and Large Language Models (LLMs) has led to groundbreaking advancements across various fields. This essay explores the symbiotic relationship between these two technologies, delving into their individual capabilities and the transformative impact they have when combined. Through examining their applications in creative endeavors, problem-solving, and knowledge dissemination, it becomes evident that their synergy heralds a new era of innovation and comprehension.

Understanding Generative AI and Large Language Models:
Generative AI refers to a subset of artificial intelligence that focuses on generating new content autonomously. It encompasses a wide range of techniques, from rule-

In [46]:
print(response['review'])




Here are 5 multiple-choice questions based on the text 'The Synergy of Generative AI and Large Language Models: A Paradigm Shift in Creativity and Understanding':
1. Within the field of ____________, Generative AI can generate novel molecular structures with desired properties.
A. Creativity 
B. Art 
C. Drug Discovery 
D. Design 

2. Which of the following best describes the core purpose of Large Language Models?
A. To enhance human-computer interactions 
B. To innovate creative expression 
C. To synthesize and summarize vast amounts of information 
D. To assist in decision-making processes 

3. Which of the following applications is MOST aligned with the synergy between Generative AI and Large Language Models?
A. Recommendation Systems 
B. Virtual Assistants 
C. Machine Translation 
D. Automated Essay Scoring 

4. Generative AI can have a notable impact on which of the following areas within the healthcare sector?
A. Diagnosis and Treatment 
B. Patient Monitoring 
C. Clinical Decision

In [39]:
llm2 = ChatCohere(model="command", max_tokens=256, temperature=0)

In [40]:
template = '''
 convert the given string into dictionary format.{string}


'''
json_prompt = PromptTemplate(
    input_variables=["string"],
    template=template
)

json_chain=LLMChain(llm=llm2, prompt=json_prompt, verbose=True)

In [41]:
json_op= json_chain.run(response.get('quiz'))



> Entering new LLMChain chain...
Prompt after formatting:

 convert the given string into dictionary format.Here are 5 multiple-choice questions based on the text "The Synergy of Generative AI and Large Language Models: A Paradigm Shift in Creativity and Understanding" for students of Artificial Intelligence: 

1. 
Multiple Choice Question: Generative AI utilizes techniques such as ________ and ________ to generate new content. 
Options:
A Rule-based Systems and Deep Learning
B Statistical Analysis and Rule-based Systems
C Deep Learning and Large Language Models
D Statistical Properties and Deep Learning

Answer:
D 

2. 
Multiple Choice Question: Large Language Models are trained on massive datasets to learn the ________ of language. 
Options:
A Rules
B Coherence
C Correlation
D Statistics

Answer:
D 

3. 
Multiple Choice Question: The synergy between Generative AI and Large Language Models has enhanced creativity and innovation in ________ domains. 
Options:
A Art and Design
B Journ

In [24]:
json_op = json_op.replace("```json ","")
json_op = json_op.replace("```","")

In [42]:
json_op

'Here is the provided text converted to dictionary format:\n\n```python\nartificial_intelligence = {\n"title": "The Synergy of Generative AI and Large Language Models: A Paradigm Shift in Creativity and Understanding",\n"questions": [\n{"question": "Which option best describes the type of content generation used by Generative AI?",\n "options": ["A Rule-based Systems and Deep Learning", "B Statistical Analysis and Rule-based Systems", "C Deep Learning and Large Language Models", "D Statistical Properties and Deep Learning"],\n "answer": "D"},\n {"question": "What do Large Language Models learn during training?", "options": ["A Rules", "B Coherence", "C Correlation", "D Statistics"], "answer": "D"},\n {"question": "Which domains have benefited the most from the synergy between Generative AI and Large Language Models?", "options": ["A Art and Design", "B Journalism", "C Healthcare", "D Engineering"], "answer": "A"},\n {"question": "Which aspect of content creation can Generative AI and L

In [43]:
print(json_op)

Here is the provided text converted to dictionary format:

```python
artificial_intelligence = {
"title": "The Synergy of Generative AI and Large Language Models: A Paradigm Shift in Creativity and Understanding",
"questions": [
{"question": "Which option best describes the type of content generation used by Generative AI?",
 "options": ["A Rule-based Systems and Deep Learning", "B Statistical Analysis and Rule-based Systems", "C Deep Learning and Large Language Models", "D Statistical Properties and Deep Learning"],
 "answer": "D"},
 {"question": "What do Large Language Models learn during training?", "options": ["A Rules", "B Coherence", "C Correlation", "D Statistics"], "answer": "D"},
 {"question": "Which domains have benefited the most from the synergy between Generative AI and Large Language Models?", "options": ["A Art and Design", "B Journalism", "C Healthcare", "D Engineering"], "answer": "A"},
 {"question": "Which aspect of content creation can Generative AI and Large Languag

In [29]:
qz = json.loads(json_op)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [27]:
import datetime

# Get the current date and time
now = datetime.datetime.now()

# Convert the datetime object to a string
date_time_str = now.strftime("%Y-%m-%d %H:%M:%S")



In [28]:
filename = "mcqs"+ now.strftime("%Y-%m-%d-%H-%M")

with open(filename, 'w') as f:
    f.write(response.get('quiz'))